In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)

# data_path = 'data/twitter/washed_data.csv'
data_path = 'data/twitter/unwashed_data.csv'
df = pd.read_csv(data_path)

# to numpy
X=df['datas'].values
y=df['labels'].values
print(type(X))
print('length of X:',len(X))

shuffle_index = np.random.permutation(len(X))
X, y = X[shuffle_index], y[shuffle_index]

n_samples = 10000

X = X[:n_samples]
y = y[:n_samples]
print('length of X:',len(X))

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# to list
X_train = X_train.tolist()
X_test = X_test.tolist()

print('length of X_train:',len(X_train))
print('length of X_test:',len(X_test))

<class 'numpy.ndarray'>
length of X: 1600000
length of X: 10000
length of X_train: 8000
length of X_test: 2000


In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments

# model_path = 'model/gpt2'
model_path = 'gpt2'

# 加载分词器和模型
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# padding
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

print("Tokenizer pad token:", tokenizer.pad_token)
print("Model pad token ID:", model.config.pad_token_id)


# 数据预处理
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=256)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizer pad token: <|endoftext|>
Model pad token ID: 50256


In [4]:
# 准备数据集
class GPT2Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GPT2Dataset(train_encodings, y_train)
val_dataset = GPT2Dataset(val_encodings, y_test)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 训练轮次
    per_device_train_batch_size=8,   # 每个设备的批次大小
    per_device_eval_batch_size=16,   # 每个设备的评估批次大小
    warmup_steps=500,                # 预热步数
    learning_rate=1e-5,              # 学习率
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
    logging_steps=10,
)

# 初始化训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 先使用未训练的模型进行评估
predictions = trainer.predict(val_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = y_test
print("Accuracy:", accuracy_score(y_true, y_pred))
Precision, Recall, F1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print("Precision:", Precision)
print("Recall:", Recall)
print("F1:", F1)

# print(y_pred)


100%|██████████| 125/125 [00:16<00:00,  7.57it/s]wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xuy22128. Use `wandb login --relogin` to force relogin


100%|██████████| 125/125 [00:24<00:00,  5.18it/s]
/Users/xuy/anaconda3/envs/xuy/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.49
Precision: 0.0
Recall: 0.0
F1: 0.0
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x306dd8910>> (for post_run_cell), with arguments args (<ExecutionResult object at 306e0a310, execution_count=5 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 306e0a450, raw_cell="from sklearn.metrics import accuracy_score, precis.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/xuy/Documents/Study/%E5%A4%A7%E4%B8%89%E4%B8%8B/%E5%B7%A5%E7%A7%91%E5%88%9B4J/CS3507/sentiment_analysis.ipynb#W4sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [6]:
# 开始训练
trainer.train()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x306dd8910>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 30843be10, raw_cell="# 开始训练
trainer.train()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/xuy/Documents/Study/%E5%A4%A7%E4%B8%89%E4%B8%8B/%E5%B7%A5%E7%A7%91%E5%88%9B4J/CS3507/sentiment_analysis.ipynb#W5sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

  0%|          | 10/3000 [00:05<25:25,  1.96it/s] 

{'loss': 2.0685, 'grad_norm': 81.24900817871094, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.01}


  1%|          | 20/3000 [00:10<22:57,  2.16it/s]

...

100%|██████████| 3000/3000 [23:14<00:00,  2.15it/s]

{'train_runtime': 1394.6733, 'train_samples_per_second': 17.208, 'train_steps_per_second': 2.151, 'train_loss': 0.4816983826160431, 'epoch': 3.0}


TrainOutput(global_step=3000, training_loss=0.4816983826160431, metrics={'train_runtime': 1394.6733, 'train_samples_per_second': 17.208, 'train_steps_per_second': 2.151, 'total_flos': 1837242777600000.0, 'train_loss': 0.4816983826160431, 'epoch': 3.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x306dd8910>> (for post_run_cell), with arguments args (<ExecutionResult object at 307780210, execution_count=6 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 30843be10, raw_cell="# 开始训练
trainer.train()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/xuy/Documents/Study/%E5%A4%A7%E4%B8%89%E4%B8%8B/%E5%B7%A5%E7%A7%91%E5%88%9B4J/CS3507/sentiment_analysis.ipynb#W5sZmlsZQ%3D%3D> result=TrainOutput(global_step=3000, training_loss=0.4816983826160431, metrics={'train_runtime': 1394.6733, 'train_samples_per_second': 17.208, 'train_steps_per_second': 2.151, 'total_flos': 1837242777600000.0, 'train_loss': 0.4816983826160431, 'epoch': 3.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [ ]:
# 评估模型
predictions = trainer.predict(val_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = y_test
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision, Recall, F1:", precision_recall_fscore_support(y_true, y_pred, average='binary'))


In [ ]:
# 保存模型
model_path = 'fine_tuned_models/gpt2-twitter'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)